In [ ]:
from PIL import Image
import os
from diffusers import DiffusionPipeline, DDIMScheduler
import torch
import numpy as np

# --- SETTINGS ---
adapters = ["ikea", "cereal", "toy-face"]
output_dir = "lora_grid_fast"
os.makedirs(output_dir, exist_ok=True)

# Load pipeline
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

# Load adapters
pipe.load_lora_weights("ostris/ikea-instructions-lora-sdxl", adapter_name="ikea")
pipe.load_lora_weights("ostris/super-cereal-sdxl-lora", adapter_name="cereal")
pipe.load_lora_weights("CiroN2022/toy-face", adapter_name="toy-face")

# Define 14 handpicked combinations (spread across triangle)
weights_list = 
    [1.0, 0.0, 0.0],
    [0.1, 0.0, 0.0],
    [0.1, 0.0, 0.0],
    [0.1, 0.0, 0.0]


# Generate images
images = []
for weights in weights_list:
    pipe.set_adapters(adapters, adapter_weights=weights)
    prompt = "a man"
    image = pipe(prompt=prompt, height=512, width=512, num_inference_steps=10).images[0]
    images.append(image)

# Make 2×7 grid
grid_width = 7
grid_height = 2
img_w, img_h = images[0].size
grid_img = Image.new("RGB", (img_w * grid_width, img_h * grid_height))

for idx, img in enumerate(images):
    x = (idx % grid_width) * img_w
    y = (idx // grid_width) * img_h
    grid_img.paste(img, (x, y))

grid_path = os.path.join(output_dir, "lora_grid_2x7.png")
grid_img.save(grid_path)
print("✅ Grid saved to:", grid_path)


/root/stable_diffusion/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.51it/s]
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you thi

✅ Grid saved to: lora_grid_fast/lora_grid_2x7.png
